In [ ]:
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os
import pickle
import requests
from PIL import Image
from IPython.display import display
import torch

# import and set Processor and Model

from transformers import Blip2Processor, Blip2ForConditionalGeneration

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b")



pytorch_model-00001-of-00002.bin:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.50G [00:00<?, ?B/s]

In [ ]:
# defined captioning function

def caption(img):
    raw_image = img.convert('RGB')
    inputs = processor(raw_image, return_tensors="pt")

    out = model.generate(**inputs)
    return processor.decode(out[0], skip_special_tokens=True)

In [ ]:
#set location of client secret file you got from Google Cloud Console and required API scopes

CLIENT_SECRET = 'client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/photoslibrary']

In [ ]:
#authenticate and authorize, save token in a .pickle

creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)

if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            CLIENT_SECRET, SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)


In [ ]:
#get all media item info

response = requests.get('https://photoslibrary.googleapis.com/v1/mediaItems', headers={'Authorization': f'Bearer {creds.token}'})
mediaItems = response.json()['mediaItems']

#loop through all the media items to get each base url and feed it to the captioning model function

baseUrls = []

for i in mediaItems:
    # baseUrls.append(i['baseUrl'])
    img = Image.open(requests.get(i['baseUrl'], stream=True).raw)
    display(img)
    print(caption(img))
